In [1]:
from mtcnn.mtcnn import MTCNN
from fastbook import *
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
from ipywidgets import *
from PIL import Image

C:\Users\SHARAD\anaconda3\lib\site-packages\fastbook\__init__.py:19: UserWarning: Missing `graphviz` - please run `conda install fastbook`
  except ModuleNotFoundError: warn("Missing `graphviz` - please run `conda install fastbook`")


In [2]:
# import pathlib
# temp = pathlib.PosixPath
# pathlib.PosixPath = pathlib.WindowsPath

In [4]:
learn_inf = load_learner('../MaskDetector/res/mask_detector_resnet18.pkl',cpu=True)

In [7]:
def on_data_change2(change):
    lbl_pred.value = ''
    face=''
    img1 = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img1.to_thumb(256,256))
    for name, file_info in btn_upload.value.items():
        img2 = Image.open(io.BytesIO(file_info['content']))
    data = np.asarray(img2)
    image = cv2.cvtColor(data, cv2.COLOR_BGR2RGB)
    detector=MTCNN()
    face=detector.detect_faces(data)
    if face:
        #lbl_pred.value = 'Humans Detected'
        pred,pred_idx,probs = learn_inf.predict(img1)
        if(pred=='mask' and probs[pred_idx] < 0.9):
            pred='no mask'
            probs[pred_idx]=1-probs[pred_idx]
        lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'        
    else:
        lbl_pred.value = 'No Humans Detected'


In [8]:
xx =widgets.HTML(
    value="""
    <h1><b>Mask Detector</b></h1>
    <p>Detect whether a person is wearing a mask or not. Also takes care of edge cases where there's no human face in the picture.</p>
    """
)
btn_upload = widgets.FileUpload(accept='.jpg,.jpeg',multiple=False)
out_pl = widgets.Output()
lbl_pred = widgets.Label()
btn_upload.observe(on_data_change2, names=['data'])
display(VBox([xx,widgets.Label('Upload the Photo!'), btn_upload, out_pl, lbl_pred]))